<a href="https://colab.research.google.com/github/thor4/neuralnets/blob/master/projects/1-CNN/model_evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Steps to reproduce `model.evaluate()` error

### 1: Setup the model

This model was created using Tensorflow's transfer learning tutorial. MobileNetV2 was used as the convolutional base. Gabors in two classes representing tilts from 45 degrees in the clockwise direction and tilts in the counterclockwise direction were used to train the final classification layer. Finally, the model was fine-tuned by training the weights for layers 101 onwards.

Run the cell to download a zip file from OSF then extract its contents into the newly created directory: `content/18kim_range_ft/`

In [7]:
# @title Download model

import requests, os
from zipfile import ZipFile

print("Start downloading and unzipping `Range model`...")
name = '18kim_range_ft'
fname = f"{name}.zip"
url = f"https://osf.io/vbwsk/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall() #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `Range model`...
Download completed.


#### Load the model
Next, we load the model using Tensorflow

In [8]:
import tensorflow as tf 
model = tf.keras.models.load_model('18kim_range_ft') #load previously trained fine-tuned range model (model 2)
model.summary() #verify architecture, trainable: 2,225,153 params between last 100 layers

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281  

### 2: Download & load datasets to test model with
Download the test datasets from OSF and extract the contents into the newly created directory: `content/datasets/`

In [9]:
print("Start downloading and unzipping `18 test datasets`...")
name = 'model2_dset1-18_cond'
fname = f"{name}.zip"
url = f"https://osf.io/jkryf/download" #osf share link
r = requests.get(url, allow_redirects=True)
with open(fname, 'wb') as fh:
  fh.write(r.content) #download file

with ZipFile(fname, 'r') as zfile:
  zfile.extractall("datasets") #extract contents

if os.path.exists(fname):
  os.remove(fname) #delete zip file
else:
  print(f"The file {fname} does not exist")

print("Download completed.")

Start downloading and unzipping `18 test datasets`...
Download completed.


Load all 18 sets and use prefetch to streamline image loading

In [13]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32 
IMG_SIZE = (160, 160) #forces a resize from 170x170 since MobileNetV2 has weights only for certain sizes
AUTOTUNE = tf.data.AUTOTUNE #prompts the tf.data runtime to tune the value dynamically at runtime
def model2_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE):
    curr_dir = os.getcwd() 
    set1_dir = os.path.join(curr_dir, 'datasets/s1-t_0.1-c_0.3')
    set2_dir = os.path.join(curr_dir, 'datasets/s2-t_0.1-c_0.45')
    set3_dir = os.path.join(curr_dir, 'datasets/s3-t_0.1-c_1')
    set4_dir = os.path.join(curr_dir, 'datasets/s4-t_0.2-c_0.3')
    set5_dir = os.path.join(curr_dir, 'datasets/s5-t_0.2-c_0.45')
    set6_dir = os.path.join(curr_dir, 'datasets/s6-t_0.2-c_1')
    set7_dir = os.path.join(curr_dir, 'datasets/s7-t_0.4-c_0.3')
    set8_dir = os.path.join(curr_dir, 'datasets/s8-t_0.4-c_0.45')
    set9_dir = os.path.join(curr_dir, 'datasets/s9-t_0.4-c_1')
    set10_dir = os.path.join(curr_dir, 'datasets/s10-t_0.8-c_0.3')
    set11_dir = os.path.join(curr_dir, 'datasets/s11-t_0.8-c_0.45')
    set12_dir = os.path.join(curr_dir, 'datasets/s12-t_0.8-c_1')
    set13_dir = os.path.join(curr_dir, 'datasets/s13-t_1.6-c_0.3')
    set14_dir = os.path.join(curr_dir, 'datasets/s14-t_1.6-c_0.45')
    set15_dir = os.path.join(curr_dir, 'datasets/s15-t_1.6-c_1')
    set16_dir = os.path.join(curr_dir, 'datasets/s16-t_3.2-c_0.3')
    set17_dir = os.path.join(curr_dir, 'datasets/s17-t_3.2-c_0.45')
    set18_dir = os.path.join(curr_dir, 'datasets/s18-t_3.2-c_1')
    set1 = image_dataset_from_directory(set1_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE) #3000 images 2 classes
    set2 = image_dataset_from_directory(set2_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set3 = image_dataset_from_directory(set3_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set4 = image_dataset_from_directory(set4_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set5 = image_dataset_from_directory(set5_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set6 = image_dataset_from_directory(set6_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set7 = image_dataset_from_directory(set7_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE) #3000 images 2 classes
    set8 = image_dataset_from_directory(set8_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set9 = image_dataset_from_directory(set9_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set10 = image_dataset_from_directory(set10_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set11 = image_dataset_from_directory(set11_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set12 = image_dataset_from_directory(set12_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set13 = image_dataset_from_directory(set13_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE) #3000 images 2 classes
    set14 = image_dataset_from_directory(set14_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set15 = image_dataset_from_directory(set15_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set16 = image_dataset_from_directory(set16_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set17 = image_dataset_from_directory(set17_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set18 = image_dataset_from_directory(set18_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)
    set2 = set2.prefetch(buffer_size=AUTOTUNE) 
    set1 = set1.prefetch(buffer_size=AUTOTUNE) 
    set3 = set3.prefetch(buffer_size=AUTOTUNE) 
    set4 = set4.prefetch(buffer_size=AUTOTUNE) 
    set5 = set5.prefetch(buffer_size=AUTOTUNE) 
    set6 = set6.prefetch(buffer_size=AUTOTUNE) 
    set7 = set7.prefetch(buffer_size=AUTOTUNE) 
    set8 = set8.prefetch(buffer_size=AUTOTUNE) 
    set9 = set9.prefetch(buffer_size=AUTOTUNE) 
    set10 = set10.prefetch(buffer_size=AUTOTUNE)
    set11 = set11.prefetch(buffer_size=AUTOTUNE)
    set12 = set12.prefetch(buffer_size=AUTOTUNE)
    set13 = set13.prefetch(buffer_size=AUTOTUNE)
    set14 = set14.prefetch(buffer_size=AUTOTUNE)
    set15 = set15.prefetch(buffer_size=AUTOTUNE)
    set16 = set16.prefetch(buffer_size=AUTOTUNE)
    set17 = set17.prefetch(buffer_size=AUTOTUNE)
    set18 = set18.prefetch(buffer_size=AUTOTUNE)
    return set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16,set17,set18

set1,set2,set3,set4,set5,set6,set7,set8,set9,set10,set11,set12,set13,set14,set15,set16,set17,set18 = model2_init_sets(BATCH_SIZE, IMG_SIZE, AUTOTUNE)

Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


### 3: Test different datasets
Setup and run the test to calculate both Tf's accuracy and my naive calculation

In [14]:
def test_dataset(current_set):
  all_acc=tf.zeros([], tf.float64) #initialize array to hold all accuracy indicators (single element)
  loss, acc = model.evaluate(current_set) #now test the model's performance on the test set
  for image_batch, label_batch in current_set.as_numpy_iterator():
      predictions = model.predict_on_batch(image_batch).flatten() #run batch through model and return logits
      predictions = tf.nn.sigmoid(predictions) #apply sigmoid activation function to transform logits to [0,1]
      predictions = tf.where(predictions < 0.5, 0, 1) #round down or up accordingly since it's a binary classifier
      accuracy = tf.where(tf.equal(predictions,label_batch),1,0) #correct is 1 and incorrect is 0
      all_acc = tf.experimental.numpy.append(all_acc, accuracy)
  all_acc = all_acc[1:]  #drop first placeholder element
  avg_acc = tf.reduce_mean(all_acc)
  print('My Accuracy:', avg_acc.numpy()) 
  print('Tf Accuracy:', acc) 


In [15]:
test_dataset(set1)

32/32 [==============================] - 16s 451ms/step - loss: 0.6922 - accuracy: 0.5030
My Accuracy: 0.543
Tf Accuracy: 0.503000020980835


My accuracy yielded a 54.3% accuracy while Tensorflow's yielded 50.30%

In [16]:
test_dataset(set4)

32/32 [==============================] - 15s 449ms/step - loss: 0.6854 - accuracy: 0.5040
My Accuracy: 0.558
Tf Accuracy: 0.5040000081062317


My accuracy yielded a 54.3% accuracy while Tensorflow's yielded 50.30%

In [17]:
test_dataset(set10)

32/32 [==============================] - 14s 439ms/step - loss: 0.6817 - accuracy: 0.5040
My Accuracy: 0.552
Tf Accuracy: 0.5040000081062317


My accuracy yielded a 55.2% accuracy while Tensorflow's yielded 50.40%

In [18]:
test_dataset(set15)

32/32 [==============================] - 14s 439ms/step - loss: 0.1042 - accuracy: 0.9660
My Accuracy: 0.987
Tf Accuracy: 0.9660000205039978


My accuracy yielded a 98.7% accuracy while Tensorflow's yielded 96.6%

In [19]:
test_dataset(set16)

32/32 [==============================] - 14s 434ms/step - loss: 0.6551 - accuracy: 0.5200
My Accuracy: 0.648
Tf Accuracy: 0.5199999809265137


My accuracy yielded a 64.8% accuracy while Tensorflow's yielded ~52%

In [20]:
test_dataset(set17)

32/32 [==============================] - 15s 451ms/step - loss: 0.5398 - accuracy: 0.6750
My Accuracy: 0.832
Tf Accuracy: 0.675000011920929


My accuracy yielded a 83.2% accuracy while Tensorflow's yielded 67.5%

In [22]:
test_dataset(set18)

32/32 [==============================] - 15s 457ms/step - loss: 0.0091 - accuracy: 1.0000
My Accuracy: 1.0
Tf Accuracy: 1.0


My accuracy yielded 100% accuracy while Tensorflow's yielded 100%

Dataset 17 showed the biggest disparity and dataset 18 was the only set that showed the same accuracy in both calculations